In [1]:
#read metadata such as subject ages etc
import numpy as np
import pandas as pd
import os
from datetime import datetime
from docx import Document
sites = ['Aalto','Tuebingen']
source_path_base =r"D:\REFTEP_ALL\Source_analysis"
check_mri_t2 = True
check_coil_controls = True

In [2]:
if check_mri_t2:
    n_subs = 0
    n_t2_images = 0
    for site in sites:
        source_path_site = os.path.join(source_path_base,f'Source_analysis_{site}')
        subjects_dir = fr"D:\REFTEP_ALL\REFTEP_reco\{site}_recon_all/"
        for subject in os.listdir(source_path_site):
            subject_reco = subject[0:7] + "_reco"
            n_subs +=1
            if os.path.exists(os.path.join(subjects_dir,subject_reco,'mri','T2.mgz')):
                n_t2_images +=1
    print(f'found recon all dir for {n_subs} but T2 images only for {n_t2_images} subjects.')

found recon all dir for 51 but T2 images only for 49 subjects.


In [ ]:
#upper code cell computing these variables cell has been deleted
print("men",men)
print("women",women)
print("ages (mean pm std)",mean_ages,sds_ages)
print("rmts (mean pm std)",mean_rmts,sds_rmts)
print(f"men total:{np.sum(list(men.values()))}")
print(f"women total:{np.sum(list(women.values()))}")
agelist = []
for site in sites:
    agelist = agelist + ages[site]
print(f"ages total (mean pm std): {round(np.mean(agelist),2)}, {round(np.std(agelist),2)}")


men {'Aalto': 13, 'Tuebingen': 10}
women {'Aalto': 10, 'Tuebingen': 18}
ages (mean pm std) {'Aalto': 27.6, 'Tuebingen': 27.4} {'Aalto': 6.9, 'Tuebingen': 5.1}
rmts (mean pm std) {'Aalto': 47.1, 'Tuebingen': 51.4} {'Aalto': 7.8, 'Tuebingen': 8.9}
men total:23
women total:28
ages total (mean pm std): 27.49, 5.96


In [5]:
#read coil control parameters
import mne
import scipy
def get_angle_differences(directions, target_direction):
    angle_list = []
    for compare in directions:
        dotprod = np.dot(compare,target_direction)
        magnitude_ori = np.linalg.norm(compare)
        magnitude_target_ori = np.linalg.norm(target_direction)
        angle = np.degrees(np.arccos(np.clip(dotprod/(magnitude_ori*magnitude_target_ori),-1.0,1.0)))
        angle_list.append(angle)
    return np.array(angle_list)

def read_coil_control_2(coil_control_path,n_trials,pos_diff_default,angle_diff_default,distance_thresh, angle_distance_thresh):
    legit_coil_pos = True
    try:
        if "Aalto" in coil_control_path:
            target_name = "target_marker"
        elif 'Tuebingen' in coil_control_path:
            target_name = "instrument_marker"
        coil_control = scipy.io.loadmat(coil_control_path,simplify_cells=True)['trigger_markers_remaining']['trigger_markers']
        target = scipy.io.loadmat(coil_control_path,simplify_cells=True)['trigger_markers_remaining'][target_name]
        good_inds_notnan = np.array([i for i in range(len(coil_control)) if str(coil_control[i]['stimulus_id'])!='nan']) #some stimuli are not detected

        if 'sub-020' in coil_control_path: #here the coil control was constantly off, maybe bad instrument marker,...so using median target
            target_pos = np.median([np.array(coil_control[i]['coil_pos']) for i in good_inds_notnan],axis=0)
            target_normal = np.median([np.array(coil_control[i]['coil_normal']) for i in good_inds_notnan],axis=0)
            target_ori = np.median([np.array(coil_control[i]['coil_dir']) for i in good_inds_notnan],axis=0)
        else:
            target_pos = target['coil_pos']
            target_normal = target['coil_normal']
            target_ori = target['coil_dir']
        ids = [np.array(coil_control[i]['stimulus_id']) for i in range(len(coil_control)) if i in good_inds_notnan]
        if len(ids) != len(np.unique(ids)):
            print(ids)
            print("failed")
        #get coil pos, ori and normal with dummy values for nan values
        coil_positions_all = [np.array(coil_control[i]['coil_pos']) if i in good_inds_notnan else np.array(target_pos) for i in range(len(coil_control))]
        coil_oris_all = [np.array(coil_control[i]['coil_dir']) if i in good_inds_notnan else np.array(target_ori) for i in range(len(coil_control))]
        coil_normals_all = [np.array(coil_control[i]['coil_normal']) if i in good_inds_notnan else np.array(target_normal) for i in range(len(coil_control))]
        #print(target_pos, target_normal, target_ori, coil_positions_all[0], coil_oris_all[0], coil_normals_all[0])
        distances_poses = np.linalg.norm(coil_positions_all - np.array(target_pos),axis=1) #distances of position to the target marker of each stim
        good_indices_distances_poses = list(np.where(distances_poses <= distance_thresh)[0])

        #angle differences to the target and apply thresholding
        distances_oris = get_angle_differences(coil_oris_all, target_ori)
        good_indices_distances_oris = list(np.where(distances_oris <= angle_distance_thresh)[0])

        distances_normals = get_angle_differences(coil_normals_all, target_normal)
        good_indices_distances_normals = list(np.where(distances_normals <= angle_distance_thresh)[0])

        good_indices_distances = np.intersect1d(np.intersect1d(good_indices_distances_oris, good_indices_distances_normals),good_indices_distances_poses)
        if len(good_indices_distances) < len(distances_poses):
            print(f"Dropping {len(good_indices_distances) - len(distances_poses)} trials for {subject} based on coil deviation")

        good_inds = np.intersect1d(good_indices_distances,good_inds_notnan).astype(int)
        median_diff_pos = np.median(distances_poses[good_inds_notnan])
        median_diff_normal = np.median(distances_normals[good_inds_notnan])
        median_diff_ori = np.median(distances_oris[good_inds_notnan])
        diff_pos = np.array([dist if ind in good_inds else median_diff_pos for ind, dist in enumerate(distances_poses)])
        diff_normal = np.array([dist if ind in good_inds else median_diff_normal for ind, dist in enumerate(distances_normals)])
        diff_ori = np.array([dist if ind in good_inds else median_diff_ori for ind, dist in enumerate(distances_oris)])

    except FileNotFoundError:
        legit_coil_pos = False
        good_inds_notnan = None
        good_inds = None
        #if the file does not exist then use zero difference for ori and normal but 1.5 (mm) difference for pos
        #print(f"Using default difference values of position diff: {pos_diff_default} mm and normal and ori: {angle_diff_default} degrees as {coil_control_path} was not found or was otherways marked bad.")
        diff_pos = np.ones(shape=(1,n_trials))[0]*pos_diff_default
        diff_normal = np.ones(shape=(1,n_trials))[0]*angle_diff_default
        diff_ori = np.ones(shape=(1,n_trials))[0]*angle_diff_default
        good_indices_distances = np.arange(n_trials)
    return diff_pos, diff_normal, diff_ori, good_indices_distances, good_inds_notnan, good_inds, legit_coil_pos #return the coil control parameters

if check_coil_controls:
    n_rejected_trials = []
    n_total_trials = []
    n_trials_nan = []
    no_legit_coil_poses = 0
    pos_diff_default = 2.0
    angle_diff_default = 1.6
    distance_thresh = np.inf
    angle_distance_thresh = np.inf
    rejtrials = 0
    diff_poses = []
    n_nan_trials_all = []
    n_total_trials_numbers = 0
    diff_oris = {'coil_normal':[],'coil_dir':[]}
    feature_path_base = r"D:\REFTEP_ALL\Features_v2"
    if check_coil_controls:
        for site in sites:
            features_path_site = os.path.join(feature_path_base,f'Features_{site}')
            source_path_site = os.path.join(source_path_base,f'Source_analysis_{site}')
            for subject in os.listdir(features_path_site):
                subject_source_path = os.path.join(source_path_site,subject)
                n_trials = mne.read_epochs(os.path.join(subject_source_path,f'{subject}_final_eeg-epo.fif'),verbose=False).get_data(copy=True).shape[0]
                #print(subject)
                coil_control_path = os.path.join(features_path_site,subject,f'{subject}_2stimulations_final.mat')
                diff_pos, diff_normal, diff_ori, good_indices_distances, good_inds_notnan, good_inds, legit_coil_pos = read_coil_control_2(coil_control_path,n_trials,pos_diff_default,angle_diff_default,distance_thresh,angle_distance_thresh)
                if len(diff_pos) != n_trials:
                    raise ValueError(f"bad number of trials compared to coil control {n_trials} vs. {len(diff_pos)}")
                if legit_coil_pos:
                    rejtrials += n_trials - len(good_indices_distances)
                    n_total_trials.append(n_trials)
                    n_rejected_trials.append(100*(n_trials - len(good_indices_distances))/n_trials)
                    n_trials_nan.append(100*(n_trials - len(good_inds_notnan))/n_trials)
                    diff_oris['coil_normal'].append(np.mean(diff_normal[good_inds]))
                    diff_oris['coil_dir'].append(np.mean(diff_ori[good_inds]))
                    diff_poses.append(np.mean(diff_pos[good_inds]))
                    n_nan_trials_all.append(n_trials-len(good_inds_notnan))
                else:
                    no_legit_coil_poses += 1
                    n_nan_trials_all.append(n_trials)
                n_total_trials_numbers += n_trials
    print(f"trials rejected with deviation {rejtrials}")
    print(f'position rejection: {round(np.mean(n_rejected_trials),1)} ± {round(np.std(n_rejected_trials),1)}')
    print(f'number of nan stimuli to be replaced by individual info: {round(np.mean(n_trials_nan),1)} ± {round(np.std(n_trials_nan),1)}')
    print(f'no legit coil poses: {no_legit_coil_poses}')

    print("targeting accuracies")
    print(f'position mean std: {round(np.mean(diff_poses),1)} ± {round(np.std(diff_poses),1)}')
    print(f'position median: {np.median(diff_poses)}')
    print(f'total nan trials to be replaced {100*np.sum(n_nan_trials_all)/n_total_trials_numbers}')
    print(f'{list(diff_oris.keys())[0]}: {round(np.mean(diff_oris[list(diff_oris.keys())[0]]),1)} ± {round(np.std(diff_oris[list(diff_oris.keys())[0]]),1)}')
    print(f'{list(diff_oris.keys())[1]}: {round(np.mean(diff_oris[list(diff_oris.keys())[1]]),1)} ± {round(np.std(diff_oris[list(diff_oris.keys())[1]]),1)}')

                


trials rejected with deviation 0
position rejection: 0.0 ± 0.0
number of nan stimuli to be replaced by individual info: 4.7 ± 15.1
no legit coil poses: 1
targeting accuracies
position mean std: 2.3 ± 0.9
position median: 2.125563855244972
total nan trials to be replaced 6.440579146827325
coil_normal: 1.3 ± 0.8
coil_dir: 1.4 ± 0.7


In [6]:
print(f'total nan trials to be replaced {100*np.sum(n_nan_trials_all)/n_total_trials_numbers} percent')
print(f'position: {np.median(diff_poses)}')
print(f'{list(diff_oris.keys())[0]}: {np.median(diff_oris[list(diff_oris.keys())[0]])}')
print(f'{list(diff_oris.keys())[1]}: {np.median(diff_oris[list(diff_oris.keys())[1]])}')

total nan trials to be replaced 6.440579146827325 percent
position: 2.125563855244972
coil_normal: 1.1898413884798587
coil_dir: 1.2484358972410208
